# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    
from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils, ml_methods_utils
from planning_analysis.show_planning import nxt_ff_utils, show_planning_utils
from neural_data_analysis.neural_analysis_tools.gpfa_methods import elephant_utils, fit_gpfa_utils, plot_gpfa_utils, gpfa_helper_class
from neural_data_analysis.neural_analysis_tools.align_trials import time_resolved_regression, time_resolved_gpfa_regression,plot_time_resolved_regression
from neural_data_analysis.neural_analysis_tools.align_trials import align_trial_utils
from decision_making_analysis.compare_GUAT_and_TAFT import find_GUAT_or_TAFT_trials

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_psth import core_stops_psth, get_stops_utils, psth_postprocessing, psth_stats, compare_events, dpca_utils
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm import binning_for_glm, stop_glm_fit, cv_stop_glm, lagged_design, stop_design


import sys
import math
import gc
import subprocess
from pathlib import Path

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi
import cProfile
import pstats

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca

# To fit gpfa
import numpy as np
from importlib import reload
from scipy.integrate import odeint
import quantities as pq
import neo
from elephant.spike_train_generation import inhomogeneous_poisson_process
from elephant.gpfa import GPFA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from elephant.gpfa import gpfa_core, gpfa_util

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

print("done")


%load_ext autoreload
%autoreload 2
%matplotlib inline

# retrieve data

In [ ]:
#raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0413"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0321"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0329"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0403"

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0312"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0316"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0327"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0328"

In [ ]:
reduce_y_var_lags = False
planning_data_by_point_exists_ok = True
y_data_exists_ok = True

pn = pn_aligned_by_event.PlanningAndNeuralEventAligned(raw_data_folder_path=raw_data_folder_path)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=planning_data_by_point_exists_ok)
# pn.planning_data_by_point, cols_to_drop = general_utils.drop_columns_with_many_nans(
#     pn.planning_data_by_point)
#pn.get_x_and_y_data_for_modeling(exists_ok=y_data_exists_ok, reduce_y_var_lags=reduce_y_var_lags)

if not hasattr(pn, 'spikes_df'):
    pn.retrieve_or_make_monkey_data()
    pn.spikes_df = neural_data_processing.make_spikes_df(pn.raw_data_folder_path, pn.ff_caught_T_sorted,
                                                            sampling_rate=pn.sampling_rate)

# NEXT: try stop end time instead

# Get captures

In [ ]:
reload(get_stops_utils)

In [ ]:
# Example wiring (mirrors your original usage)
valid_captures_df, filtered_no_capture_stops_df, stops_with_stats = get_stops_utils.prepare_no_capture_and_captures(
    monkey_information=pn.monkey_information,
    closest_stop_to_capture_df=pn.closest_stop_to_capture_df,
    ff_caught_T_new=pn.ff_caught_T_new,
    min_stop_duration=0.01,
    max_stop_duration=1.0,
    capture_match_window=0.3,
    distance_thresh=25.0,
    distance_col="distance_from_ff_to_stop",
)


In [ ]:
if 'stop_id' not in pn.closest_stop_to_capture_df.columns:
    pn.closest_stop_to_capture_df = get_stops_utils.add_stop_id_to_closest_stop_to_capture_df(
        pn.closest_stop_to_capture_df,
        pn.monkey_information,
    )

In [ ]:
pn.monkey_information.loc[17730:17740]

In [ ]:
pn.monkey_information[pn.monkey_information['monkey_speeddummy']==0].loc[17730:]

In [ ]:
pn.closest_stop_to_capture_df[pn.closest_stop_to_capture_df['stop_id'].isna()]

In [ ]:
pn.closest_stop_to_capture_df

In [ ]:
print(stops_with_stats.head(3))

# Get misses

##  one

In [ ]:
columns_to_add = ["stop_id", "stop_id_duration", "stop_id_start_time", "stop_id_end_time"]

pn.make_one_stop_w_ff_df()
one_stop_miss_df = pn.one_stop_w_ff_df[['first_stop_point_index', 'first_stop_time', 'latest_visible_ff', 'ff_distance', 'min_distance_from_adjacent_stops']].copy()
one_stop_miss_df.rename(columns={'first_stop_point_index': 'stop_point_index', 'first_stop_time': 'stop_time'}, inplace=True)
one_stop_miss_df[columns_to_add] = pn.monkey_information.loc[one_stop_miss_df['stop_point_index'], columns_to_add].values

## more

In [ ]:
pn.get_try_a_few_times_info()
pn.get_give_up_after_trying_info()


In [ ]:
columns_to_add = ["stop_id", "stop_id_duration", "stop_id_start_time", "stop_id_end_time"]
shared_columns = ["stop_point_index", "stop_time"] + columns_to_add

# --- Build expanded + ordered tables for GUAT / TAFT ---
GUAT_expanded = get_stops_utils._expand_trials(pn.GUAT_trials_df, pn.monkey_information)
TAFT_expanded = get_stops_utils._expand_trials(pn.TAFT_trials_df, pn.monkey_information)

# add stop_id to GUAT_trials_df and TAFT_trials_df
GUAT_expanded[columns_to_add] = pn.monkey_information.loc[GUAT_expanded['stop_point_index'], columns_to_add].values
TAFT_expanded[columns_to_add] = pn.monkey_information.loc[TAFT_expanded['stop_point_index'], columns_to_add].values


GUAT = get_stops_utils._add_cluster_ordering(GUAT_expanded)
TAFT = get_stops_utils._add_cluster_ordering(TAFT_expanded)

# --- Per-cluster slices (consistent, vectorized) ---
# First stop in each cluster
GUAT_first = GUAT[GUAT["is_first"]].reset_index(drop=True)
TAFT_first = TAFT[TAFT["is_first"]].reset_index(drop=True)

# Last stop in each cluster
giveup_GUAT_last = GUAT[GUAT["is_last"]].reset_index(drop=True)
capture_TAFT_last = TAFT[TAFT["is_last"]].reset_index(drop=True)

# Middle stops (exclude first and last)
GUAT_middle = GUAT[GUAT["is_middle"]].reset_index(drop=True)
TAFT_middle = TAFT[TAFT["is_middle"]].reset_index(drop=True)

# “First several” = all but the last stop in each cluster
persist_GUAT_nonfinal = GUAT[GUAT["order_in_cluster"] < GUAT["cluster_size"] - 1].reset_index(drop=True)
persist_TAFT_nonfinal = TAFT[TAFT["order_in_cluster"] < TAFT["cluster_size"] - 1].reset_index(drop=True)

# Combine the “first several” from both, keep only columns you care about, then sort by index
both_nonfinal = (
    pd.concat(
        [
            persist_GUAT_nonfinal[shared_columns],
            persist_TAFT_nonfinal[shared_columns],
        ],
        ignore_index=True
    )
    .sort_values("stop_point_index")
    .reset_index(drop=True)
)

persist_both_first = pd.concat([GUAT_first[shared_columns], 
                         TAFT_first[shared_columns]])

both_middle = pd.concat([GUAT_middle[shared_columns], 
                         TAFT_middle[shared_columns]])

# Optional: if you also want “last several” (all but the first), it’s symmetrical:
# giveup_GUAT_last_several = GUAT[GUAT["order_in_cluster"] > 0].reset_index(drop=True)
# capture_TAFT_last_several = TAFT[TAFT["order_in_cluster"] > 0].reset_index(drop=True)

giveup_GUAT_last_plus_single_miss = pd.concat([giveup_GUAT_last[shared_columns], 
                                         one_stop_miss_df[shared_columns]])

all_misses = pd.concat([one_stop_miss_df[shared_columns], 
                                         GUAT_expanded[shared_columns],
                                         persist_TAFT_nonfinal[shared_columns]
                                         ])

all_first_misses = pd.concat(
    [one_stop_miss_df[shared_columns], GUAT_first[shared_columns], TAFT_first[shared_columns]],
    ignore_index=True
)

# captures not in TAFT last (assuming capture_TAFT_last is a subset of captures)
captures_minus_TAFT_last = compare_events.diff_by(valid_captures_df, capture_TAFT_last, key='stop_id')

# non-captures excluding those flagged as 'all_misses'
non_captures_minus_all_misses = compare_events.diff_by(filtered_no_capture_stops_df, all_misses, key='stop_id')


# ===COMPARE EVENTS===

In [ ]:
# ---------- dataset registry (canonical) ----------
datasets_raw = {
    'captures': valid_captures_df.copy(),
    'no_capture': filtered_no_capture_stops_df.copy(),
    'persist_nonfinal': both_nonfinal.copy(),
    'persist_middle': both_middle.copy(),
    'giveup_GUAT_last': giveup_GUAT_last.copy(),
    'capture_TAFT_last': capture_TAFT_last.copy(),
    'giveup_single_miss': one_stop_miss_df.copy(),
    'persist_both_first': persist_both_first.copy(),
    'persist_GUAT_nonfinal': persist_GUAT_nonfinal.copy(),
    'persist_TAFT_nonfinal': persist_TAFT_nonfinal.copy(),
    'giveup_GUAT_last_plus_single_miss': giveup_GUAT_last_plus_single_miss.copy(),
    'captures_minus_TAFT_last': captures_minus_TAFT_last.copy(),
    'all_misses': all_misses.copy(),
    'non_captures_minus_all_misses': non_captures_minus_all_misses.copy(),
    'all_first_misses': all_first_misses.copy(),
}

# normalize schema + dedupe within each dataset
datasets = {k: compare_events.dedupe_within(compare_events.ensure_event_schema(v)) for k, v in datasets_raw.items()}

comparisons = compare_events.build_comparisons([
    {'a': 'captures', 'b': 'no_capture', 'key': 'captures_vs_no_capture'},

    {'a': 'persist_nonfinal', 'b': 'giveup_GUAT_last'},
    {'a': 'persist_middle',  'b': 'giveup_GUAT_last'},

    {'a': 'persist_nonfinal', 'b': 'giveup_GUAT_last_plus_single_miss'},
    {'a': 'persist_middle',   'b': 'giveup_GUAT_last_plus_single_miss'},

    {'a': 'giveup_single_miss', 'b': 'giveup_GUAT_last'},
    {'a': 'giveup_single_miss', 'b': 'persist_both_first'},

    {'a': 'persist_GUAT_nonfinal', 'b': 'persist_TAFT_nonfinal'},

    {'a': 'giveup_GUAT_last', 'b': 'capture_TAFT_last'},

    {'a': 'captures_minus_TAFT_last', 'b': 'capture_TAFT_last'},

    {'a': 'captures', 'b': 'all_misses'},

    {'a': 'non_captures_minus_all_misses', 'b': 'all_misses'},
    {'a': 'non_captures_minus_all_misses', 'b': 'all_first_misses'},
])


compare_events.validate(datasets, comparisons)


# new_seg_info

In [ ]:
stops_with_stats['stop_time'] = stops_with_stats['stop_id_start_time']
stops_with_stats['prev_time'] = stops_with_stats['stop_id_end_time'].shift(1)
stops_with_stats['next_time'] = stops_with_stats['stop_id_start_time'].shift(-1)
new_seg_info = binning_for_glm.pick_stop_window(stops_with_stats)

In [ ]:
new_seg_info

# Get bin info

In [ ]:
# 1) Build bins from your stop windows (gaps allowed)
bins_2d, meta = binning_for_glm.stops_windows_to_bins2d(new_seg_info, bin_dt=0.04, only_ok=False)

# 2) Get overlap assignments once
sample_idx, bin_idx_array, dt_array, n_bins = binning_for_glm.build_bin_assignments(
    pn.monkey_information['time'].to_numpy(),
    bins_2d,
)

# 3) Subselect raw samples once
monkey_information_sub = pn.monkey_information.iloc[sample_idx].copy()

# 3a) One pass to get exposure and used_bins
_dummy, exposure, used_bins = binning_for_glm.bin_timeseries_weighted(
    monkey_information_sub['time'].to_numpy(),  # any column of same length works
    dt_array, bin_idx_array, how='mean'
)

# 3b) Aggregate features with the SAME assignments
def agg_feat(col):
    vals = monkey_information_sub[col].to_numpy()
    out, _exp, _ub = binning_for_glm.bin_timeseries_weighted(vals, dt_array, bin_idx_array, how='mean')
    # Defensive checks: exposure/used_bins should match
    assert np.shares_memory(_exp, exposure) or np.allclose(_exp, exposure)
    assert np.array_equal(_ub, used_bins)
    return out

binned_feats = pd.DataFrame({
    'accel':     agg_feat('accel'),
    'speed':     agg_feat('speed'),
    'ang_speed': agg_feat('ang_speed'),
})

# Clean NaNs (optional: choose your policy)
binned_feats = binned_feats.replace([np.inf, -np.inf], np.nan).fillna(0.0)

# 3c) Keep bins with exposure > 0
mask_used = exposure > 0
pos = used_bins[mask_used]
offset_log = np.log(np.clip(exposure[mask_used], 1e-12, None))

binned_feats = binned_feats.iloc[mask_used].reset_index(drop=True)
binned_feats['offset_log'] = offset_log

# 4) Bin spikes per cluster across ALL bins, then slice by pos
spike_counts, cluster_ids = binning_for_glm.bin_spikes_by_cluster(
    pn.spikes_df, bins_2d, time_col='time', cluster_col='cluster'
)

# Sanity checks
assert pos.size == binned_feats.shape[0]
assert spike_counts.shape[0] >= (pos.max() + 1)

binned_spikes = pd.DataFrame(
    spike_counts[pos, :],        # slice rows to align with pos
    columns=cluster_ids,         # cluster IDs as column labels
).reset_index(drop=True)



# Add stops

In [ ]:
used_bins

In [ ]:
print(meta.head(4))

In [ ]:
meta

In [ ]:
binned_spikes

# GLM

## regular

In [ ]:
# df_X: predictors + offset (e.g., ['speed_z', 'accel_z', 'offset_log'])
# df_Y: responses per unit (e.g., columns = unit IDs)

features = ['ang_speed']
df_X = binned_feats[features + ['offset_log']]
df_Y = binned_spikes


report = stop_glm_fit.glm_mini_report(
    df_X,
    df_Y,
    offset_col='offset_log',
    feature_names=None,       # infer automatically from df_X
    cluster_ids=None,         # use df_Y.columns
    alpha=0.05,
    delta_for_rr=1.0,
    forest_term='ang_speed',         # defaults to first feature
    forest_top_n=30,
    add_intercept=True,
    cov_type='HC1',
    show_plots=True,
    save_dir=None
)


In [ ]:
cluster_id = 5
cv_stop_glm.plot_pred_vs_obs(report['results'][cluster_id], binned_feats[features], binned_spikes[cluster_id], binned_feats['offset_log'])

In [ ]:
coefs_df    = report['coefs_df']
metrics_df  = report['metrics_df']
pop_tests   = report['population_tests_df']

## CV

In [ ]:
groups = meta['stop_id']  # or your session/stop-window IDs
scores = cv_stop_glm.glm_mini_report_batch(binned_feats, binned_spikes, groups, n_splits=5)
cv_stop_glm.plot_cv_scores(scores)
scores

# new new_seg_info

In [ ]:
import numpy as np
import pandas as pd

def make_new_seg_info_from_stops(stops_with_stats: pd.DataFrame) -> pd.DataFrame:
    """
    Build the per-stop table expected by build_stop_design_from_meta from your
    stops_with_stats, carrying through any capture/reward columns if present.

    Expected columns (present in your example):
      - 'stop_id'
      - 'time' or 'stop_id_start_time' (interpreted as stop_time)
      - 'stop_id_duration' (interpreted as duration)

    Optional columns (carried through if present):
      - 'captured', 'rewarded', 'reward_size'
      - 'cond' (if you ever add one later)
      - 'trial_id' / 'session_id' (kept for your own bookkeeping)
    """
    df = stops_with_stats.copy()

    # pick stop_time from the columns you have
    if 'time' in df.columns:
        df['stop_time'] = df['time'].astype(float)
    elif 'stop_id_start_time' in df.columns:
        df['stop_time'] = df['stop_id_start_time'].astype(float)
    else:
        raise ValueError('need a stop time column: expected "time" or "stop_id_start_time"')

    # map duration
    if 'stop_id_duration' in df.columns:
        df['duration'] = df['stop_id_duration'].astype(float)
    elif {'stop_id_start_time', 'stop_id_end_time'}.issubset(df.columns):
        df['duration'] = (df['stop_id_end_time'] - df['stop_id_start_time']).astype(float)
    else:
        df['duration'] = np.nan  # ok; the helper will handle NaNs

    keep = ['stop_id', 'stop_time', 'duration']
    # carry capture/reward if present
    for opt in ['captured', 'rewarded', 'reward_size', 'cond', 'trial_id', 'session_id']:
        if opt in df.columns:
            keep.append(opt)

    new_seg_info = df[keep].drop_duplicates('stop_id').sort_values('stop_time').reset_index(drop=True)

    # sanity
    assert new_seg_info['stop_id'].is_unique, 'stop_id must be unique in new_seg_info'
    return new_seg_info

# --- usage ---------------------------------------------------------------

# 1) Build per-stop table from your stops_with_stats
new_seg_info = make_new_seg_info_from_stops(stops_with_stats)

# 2) Build the stop-aware design block (same helper we wrote earlier)
X_stop, X_stop_names = stop_design.build_stop_design_from_meta(
    meta=meta,
    pos=pos,
    new_seg_info=new_seg_info,
    speed_used=binned_feats['speed'].values,
    history_mode='gated',  # or 'single' / 'sumdiff'
    include_columns=(
        'basis', 'duration_z', 'history_gated',
        'captured', 'basis*captured',
        'rewarded_post', 'reward_size_z_post'
    )
)

# 3) Append to your existing design
X = np.column_stack([X, X_stop])


In [ ]:
new_seg_info

In [ ]:
reload(stop_design)

# stop design

## to add

Across-stop history terms (e.g., time_since_prev_stop_z): you need the ordered list of stop times, preferably as a per-stop table (new_seg_info with one row per stop). You can derive this from meta (see helper below).

Capture / reward: per-stop columns like captured (0/1), rewarded (0/1), reward_size (float). These live on new_seg_info (not in meta), then get broadcast to bins by stop_id.

## improved

In [ ]:
X_stop_df = stop_design.build_stop_design_from_meta(
    meta=meta, pos=pos, new_seg_info=new_seg_info,
    speed_used=binned_feats['speed'].values,
    history_mode='gated',
    include_columns=('basis','duration_z','history_gated','captured','basis*captured',
                     'rewarded_post','reward_size_z_post')
)
# X = np.column_stack([X, X_stop])


In [ ]:
X_stop_df

## lean model

In [ ]:
X_stop, names = build_stop_design_from_meta(
    meta=meta,
    pos=pos,
    new_seg_info=new_seg_info,
    speed_used=binned_feats['speed'].values,
    include_columns=(
        'prepost',
        'duration_z',
        'time_since_prev_stop_z',   # choose ONE: prev OR next
        'cond_dummies',
    ),
    add_interactions=False
)


## Shape-aware (captures peri-stop dynamics)

In [ ]:
X_stop, names = build_stop_design_from_meta(
    meta=meta,
    pos=pos,
    new_seg_info=new_seg_info,
    speed_used=binned_feats['speed'].values,
    include_columns=(
        'basis',                    # replaces prepost
        'duration_z',
        'time_since_prev_stop_z',   # or 'time_to_next_stop_z'
        'cond_dummies',
        'prepost*speed'             # optional; keeps a single interpretable interaction
    ),
    add_interactions=True
)


In [ ]:
# Build stop-aware block, aligned to your fitted rows
X_stop_df = stop_design.build_stop_design_from_meta(
    meta=meta,                   # the DataFrame you showed
    pos=pos,                     # global bin indices you fit (mask_used -> positions)
    new_seg_info=new_seg_info,   # per-stop table (must have stop_id, stop_time)
    speed_used=binned_feats['speed'].values, # already aligned to pos
    rc_width=0.10,               # ~200 ms support per bump
    add_interactions=True
)


In [ ]:
X_stop_df.columns

In [ ]:
assert len(binned_feats) == len(X_stop_df)
cols_to_add = [c for c in X_stop_df.columns if c not in binned_feats.columns]
binned_feats.loc[:, cols_to_add] = X_stop_df[cols_to_add].to_numpy()  # equivalent to .values


# Lagged design

## discrete lags

In [ ]:
# choose window and binning
lag_min_s, lag_max_s, bin_dt = -0.30, 0.40, 0.02
lags = lagged_design.make_integer_lags(lag_min_s, lag_max_s, bin_dt)

df_X_design, df_Y_aligned = lagged_design.build_lagged_design_by_group(
    df_X=binned_feats,
    df_Y=binned_spikes,
    group_col='stop_id',
    predictors=['speed', 'accel', 'ang_speed'],
    offset_col='offset_log',
    order_col='time',             # optional: if you have a time column
    lags_bins=lags,
    basis_df=None,
    add_intercept=True,
    standardize=True,             # z-score features
    standardize_within_group=False,
    keep_cols=['session_id']      # optional passthrough
)

results, coefs_df, metrics_df = lagged_design.fit_poisson_glm_per_cluster_df(
    df_X_design, df_Y_aligned,
    offset_col='offset_log',
    add_intercept=False,          # already added by builder
    cov_type='HC1'
)


## raised-cosine basis

In [ ]:
basis = make_raised_cosine_basis(
    n_basis=6,
    lag_min_s=-0.30,
    lag_max_s=0.40,
    bin_dt=0.02
)

df_X_basis, df_Y_aligned = build_lagged_design_by_group(
    df_X=df_X,
    df_Y=df_Y,
    group_col='stop_id',
    predictors=['speed_z', 'accel_z'],
    offset_col='offset_log',
    order_col='time',
    lags_bins=None,
    basis_df=basis,
    add_intercept=True,
    standardize=True,
    standardize_within_group=False
)

results, coefs_df, metrics_df = fit_poisson_glm_per_cluster_df(
    df_X_basis, df_Y_aligned,
    offset_col='offset_log',
    add_intercept=False,
    cov_type='HC1'
)


# Appendix

## hyper-param tuning on binwidth

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import GroupKFold

def cv_glm_per_second_loglik(X, y, offset, groups, n_splits=5):
    gkf = GroupKFold(n_splits=n_splits)
    ll, T = 0.0, 0.0  # total loglik and total time (s)
    for tr, te in gkf.split(X, y, groups):
        fit = sm.GLM(y.iloc[tr], sm.add_constant(X.iloc[tr], has_constant='add'),
                     family=sm.families.Poisson(), offset=offset.iloc[tr]).fit()
        mu = np.clip(fit.predict(sm.add_constant(X.iloc[te], has_constant='add'),
                                 offset=offset.iloc[te]), 1e-12, None)
        # Poisson log-likelihood up to constant terms
        ll += float(np.sum(y.iloc[te] * np.log(mu) - mu))
        T  += float(np.sum(np.exp(offset.iloc[te])))  # since offset=log(dt), sum dt = total seconds
    return ll / T  # loglik per second

def evaluate_bin_grid(binned_by_dt, feature_cols, target_col='spike_count',
                      offset_col='bin_dt', group_col='stop_cluster_id'):
    rows = []
    for dt, df in binned_by_dt.items():  # dict: {0.016: df16ms, 0.024: df24ms, ...}
        use = df['stop_id'].notna()
        if 'straddle_stop_boundary' in df.columns:
            use &= ~df['straddle_stop_boundary']
        X = df.loc[use, feature_cols].replace([np.inf, -np.inf], np.nan).fillna(0.0).astype(float)
        y = df.loc[use, target_col].astype(float)
        offset = np.log(df.loc[use, offset_col].astype(float))
        groups = df.loc[use, group_col] if group_col in df else pd.Series(np.arange(X.shape[0]))
        per_s = cv_glm_per_second_loglik(X, y, offset, groups)
        rows.append({'dt_s': float(dt), 'bins_per_s': 1.0/float(dt), 'cv_loglik_per_s': per_s})
    out = pd.DataFrame(rows).sort_values('dt_s')
    out['rank'] = out['cv_loglik_per_s'].rank(ascending=False, method='min').astype(int)
    return out


## Debug ff dataframe

In [ ]:
pn.make_or_retrieve_ff_dataframe()

In [ ]:
pn.ff_dataframe

In [ ]:
pn.ff_dataframe.shape

In [ ]:
h5_file_pathway = os.path.join(os.path.join(
    pn.processed_data_folder_path, 'ff_dataframe.h5'))

h5_file_pathway = 'all_monkey_data/processed_data/monkey_Schro/data_0413/ff_dataframe.h5'

ff_dataframe = pd.read_hdf(h5_file_pathway, 'ff_dataframe')
print("Retrieved ff_dataframe from", h5_file_pathway)
ff_dataframe

## use concat_new_seg_info

In [ ]:
new_seg_info['new_segment'] = np.arange(len(new_seg_info))

In [ ]:
concat_seg_data = pn_utils.concat_new_seg_info(
    pn.monkey_information, new_seg_info, bin_width=0.04)

concat_seg_data['time_since_start_time'] = concat_seg_data['time'] - concat_seg_data['new_seg_start_time']
concat_seg_data['dt'] = np.minimum(concat_seg_data['time_since_start_time'], concat_seg_data['dt'])
